In [1]:
import sqlite3
import pandas as pd
con = sqlite3.connect("proj6_readings.sqlite")
cur = con.cursor()
result = cur.execute('SELECT count(*) from readings;').fetchall()
df = pd.DataFrame(result)
df

,0
0,50000000


In [2]:
result = cur.execute('SELECT count(DISTINCT detector_id) from readings;').fetchall()
df = pd.DataFrame(result)
df.to_pickle("proj6_ex01_detector_no.pkl")
df

,0
0,146


In [3]:
result = cur.execute('SELECT detector_id, count(count), min(starttime), max(starttime) FROM readings WHERE count IS NOT NULL GROUP BY detector_id;').fetchall()
df = pd.DataFrame(result)
df.rename(columns={'0': 'detector_id', '1': 'count(count)', '2': 'min(starttime)', '3': 'max(starttime)'})
df.to_pickle("proj6_ex02_detector_stat.pkl")
df


,0,1,2,3
0,1,357214,2015-06-22 23:58:31,2016-07-12 23:57:01
1,2,357201,2015-06-22 23:58:31,2016-07-12 23:57:01
2,3,356755,2015-06-22 23:58:31,2016-07-12 23:57:01
3,4,357213,2015-06-22 23:58:31,2016-07-12 23:57:01
4,5,357204,2015-06-22 23:58:31,2016-07-12 23:57:01
...,...,...,...,...
141,142,352730,2015-06-22 23:58:31,2016-07-12 23:57:01
142,143,352729,2015-06-22 23:58:31,2016-07-12 23:57:01
143,144,352730,2015-06-22 23:58:31,2016-07-12 23:57:01
144,145,352730,2015-06-22 23:58:31,2016-07-12 23:57:01


In [4]:
result = cur.execute("""WITH readings2 AS (SELECT detector_id, count, starttime,
        LAG(count) OVER (PARTITION BY detector_id ORDER BY starttime) AS previous_count
    FROM readings
    WHERE detector_id = 146)
SELECT
    detector_id,
    count,
    previous_count
FROM readings2
LIMIT 500;""").fetchall()
df = pd.DataFrame(result)
df.rename(columns={0:'detector_id', 1:'count', 2:'prev_count'})
df.to_pickle("proj6_ex03_detector_146_lag.pkl")
df

,0,1,2
0,146,1,NaN
1,146,0,1.0
2,146,0,0.0
3,146,0,0.0
4,146,0,0.0
...,...,...,...
495,146,0,0.0
496,146,2,0.0
497,146,2,2.0
498,146,2,2.0


In [8]:
result = cur.execute("""WITH readings2 AS (SELECT detector_id, count, starttime,
        SUM(count) OVER ( PARTITION BY detector_id ORDER BY starttime ROWS BETWEEN CURRENT ROW AND 10 FOLLOWING) AS window_sum
    FROM readings
    WHERE detector_id = 146
)
SELECT
    detector_id,
    count,
    window_sum
FROM readings2
LIMIT 500;""").fetchall()
df = pd.DataFrame(result)
df.to_pickle("proj6_ex04_detector_146_sum.pkl")
df

,0,1,2
0,146,1,2
1,146,0,2
2,146,0,2
3,146,0,2
4,146,0,2
...,...,...,...
495,146,0,20
496,146,2,22
497,146,2,21
498,146,2,21
